In [1]:
import arxiv
from os import environ
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
def search_arxiv(query, max_results=10):
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.Relevance
    )
    results = []
    for result in search.results():
        results.append({
            "title": result.title,
            "summary": result.summary,
            "authors": [a.name for a in result.authors],
            "url": result.entry_id,
            "year": result.published
        })
    return results

In [8]:
papers = search_arxiv("AI", max_results=100)
for paper in papers:
    print(paper["title"], paper["url"], paper['year'])

/tmp/ipykernel_3607/644123812.py:8: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


AI Thinking: A framework for rethinking artificial intelligence in practice http://arxiv.org/abs/2409.12922v1 2024-08-26 04:41:21+00:00
Intersymbolic AI: Interlinking Symbolic AI and Subsymbolic AI http://arxiv.org/abs/2406.11563v3 2024-06-17 14:01:59+00:00
Overconfident and Unconfident AI Hinder Human-AI Collaboration http://arxiv.org/abs/2402.07632v3 2024-02-12 13:16:30+00:00
Supporting AI/ML Security Workers through an Adversarial Techniques, Tools, and Common Knowledge (AI/ML ATT&CK) Framework http://arxiv.org/abs/2211.05075v1 2022-11-09 18:07:10+00:00
Navigating Fairness: Practitioners' Understanding, Challenges, and Strategies in AI/ML Development http://arxiv.org/abs/2403.15481v2 2024-03-21 03:44:59+00:00
"Weak AI" is Likely to Never Become "Strong AI", So What is its Greatest Value for us? http://arxiv.org/abs/2103.15294v1 2021-03-29 02:57:48+00:00
A Bibliometric View of AI Ethics Development http://arxiv.org/abs/2403.05551v1 2024-02-08 16:36:55+00:00
Threefold model for AI Rea

In [9]:
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [10]:
# embeddings = OpenAIEmbeddings(openai_api_key)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/tmp/ipykernel_3607/2525196581.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
/home/celiazhang/.pyenv/versions/3.10.12/envs/arxivenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
docs = []
for paper in papers:
    content = f"title: {paper['title']} \n Summary: {paper['summary']}"
    docs.append(content)

vectorstore = Chroma.from_texts(docs, embeddings,persist_directory="../chroma_db/aidocs")

In [ ]:
# create vector database
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=30)
split_doc = text_splitter.split_documents(docs)

#
vectorstore = Chroma.from_documents(split_doc, embeddings, persist_directory=persist_dir)

In [13]:
# load vertor database
vectorstore2 = Chroma(persist_directory="../chroma_db/aidocs", embedding_function=embeddings)
# result = vectorstore.similarity_search_with_score("how to build a RAG agent")

/tmp/ipykernel_3607/834791802.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore2 = Chroma(persist_directory="../chroma_db/aidocs", embedding_function=embeddings)


In [ ]:
retriever = vectorstore2.as_retriever()

# test query
query = "Neural networks for image recognition"
retrieved_docs = retriever.get_relevant_documents(query)

for doc in retrieved_docs:
    print(doc.page_content)
    print(doc.id)
    print("\n")

title: AI-in-the-Loop -- The impact of HMI in AI-based Application 
 Summary: Artificial intelligence (AI) and human-machine interaction (HMI) are two
keywords that usually do not fit embedded applications. Within the steps needed
before applying AI to solve a specific task, HMI is usually missing during the
AI architecture design and the training of an AI model. The human-in-the-loop
concept is prevalent in all other steps of developing AI, from data analysis
via data selection and cleaning to performance evaluation. During AI
architecture design, HMI can immediately highlight unproductive layers of the
architecture so that lightweight network architecture for embedded applications
can be created easily. We show that by using this HMI, users can instantly
distinguish which AI architecture should be trained and evaluated first since a
high accuracy on the task could be expected. This approach reduces the
resources needed for AI development by avoiding training and evaluating AI
archite

In [17]:
retrieved_docs[0]

Document(metadata={}, page_content="title: AI-in-the-Loop -- The impact of HMI in AI-based Application \n Summary: Artificial intelligence (AI) and human-machine interaction (HMI) are two\nkeywords that usually do not fit embedded applications. Within the steps needed\nbefore applying AI to solve a specific task, HMI is usually missing during the\nAI architecture design and the training of an AI model. The human-in-the-loop\nconcept is prevalent in all other steps of developing AI, from data analysis\nvia data selection and cleaning to performance evaluation. During AI\narchitecture design, HMI can immediately highlight unproductive layers of the\narchitecture so that lightweight network architecture for embedded applications\ncan be created easily. We show that by using this HMI, users can instantly\ndistinguish which AI architecture should be trained and evaluated first since a\nhigh accuracy on the task could be expected. This approach reduces the\nresources needed for AI developmen

## Hugging face

In [3]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate


In [4]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)
chain = prompt | llm
question = "Who is the Pope?"

print(chain.invoke({"question": question}))
# hf_token

NameError: name 'llm' is not defined

In [2]:
hf_llm = HuggingFacePipeline(
    pipeline=pipeline("text-generation", model="DeepSeek-R1", device=0)
)

OSError: DeepSeek-R1 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`